# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [25]:
relativePath = '~/Git/python-api-challenge/WeatherPy/output_data'
worldWeather_df = pd.read_csv(relativePath + '/worldWeather.csv').drop(columns='Unnamed: 0')

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [27]:
gmaps.configure(api_key=g_key)
fig = gmaps.figure()

In [28]:
locations = worldWeather_df[['Latitude','Longitude']]
weight = worldWeather_df["Humidity"]

In [29]:
fig.add_layer(gmaps.heatmap_layer(locations,weights=weight))
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [44]:
query1 = '70 <= Max_Temperature < 80'
query2 = '`Wind Speed` < 10'
query3 = 'Clouds == 0'
queryString = query1+' and '+query2+' and '+query3

hotel_df = worldWeather_df.query(queryString)
display(hotel_df)

,City,Longitude,Latitude,Temperature,Max_Temperature,Humidity,Clouds,Wind Speed,Country,Datetime
20,alice springs,133.8833,-23.7000,73.724,73.994,23,0,5.66,AU,1619747032
21,cayenne,-52.3333,4.9333,76.226,77.000,94,0,1.11,GF,1619747033
90,jalu,21.5482,29.0331,79.196,79.196,15,0,5.53,LY,1619747077
130,huangcun,118.4037,30.0145,73.688,73.688,58,0,3.91,CN,1619747101
142,corowa,146.3833,-36.0333,71.006,71.006,44,0,0.45,AU,1619747109
167,maxixe,35.3472,-23.8597,70.574,70.574,85,0,2.32,MZ,1619747123
200,francisco sa,-43.4883,-16.4758,73.400,73.400,68,0,2.06,BR,1619747145
229,port pirie,138.0167,-33.1833,75.200,75.200,35,0,4.12,AU,1619747164
231,codrington,141.9667,-38.2667,72.158,72.158,39,0,4.66,AU,1619747165
233,hengyang,112.6150,26.8881,74.768,74.768,58,0,2.23,CN,1619747166


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [50]:
apiBase = 'https://maps.googleapis.com/maps/api/place/textsearch/json?query='

cities = hotel_df['City']
hotelNames = []

for i, city in enumerate(cities):
    try:
        apiQuery = f'lodging&location={hotel_df.iloc[i,2]},{hotel_df.iloc[i,1]}&radius=5000&key={g_key}'
        apiRequest = apiBase + apiQuery
        response = requests.get(apiRequest)
        jsonResponse = response.json()
        hotelNames.append(jsonResponse["results"][1]["name"])
    except:
        print('Error')

hotel_df['Hotel Name'] = hotelNames

C:\Users\Andrew\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


In [51]:
hotel_df

,City,Longitude,Latitude,Temperature,Max_Temperature,Humidity,Clouds,Wind Speed,Country,Datetime,Hotel Name
20,alice springs,133.8833,-23.7000,73.724,73.994,23,0,5.66,AU,1619747032,Aurora Alice Springs
21,cayenne,-52.3333,4.9333,76.226,77.000,94,0,1.11,GF,1619747033,Grand Hotel Montabo
90,jalu,21.5482,29.0331,79.196,79.196,15,0,5.53,LY,1619747077,Jalu Hotel
130,huangcun,118.4037,30.0145,73.688,73.688,58,0,3.91,CN,1619747101,September Huizhou
142,corowa,146.3833,-36.0333,71.006,71.006,44,0,0.45,AU,1619747109,Murray Rest Cottages
167,maxixe,35.3472,-23.8597,70.574,70.574,85,0,2.32,MZ,1619747123,Eclectic Beach Retreat
200,francisco sa,-43.4883,-16.4758,73.400,73.400,68,0,2.06,BR,1619747145,Pousada Francisco Sá
229,port pirie,138.0167,-33.1833,75.200,75.200,35,0,4.12,AU,1619747164,Ellen Hotel
231,codrington,141.9667,-38.2667,72.158,72.158,39,0,4.66,AU,1619747165,Hearn's Beachside Villas
233,hengyang,112.6150,26.8881,74.768,74.768,58,0,2.23,CN,1619747166,Honglin Guesthouse


In [52]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [58]:
# Add marker layer ontop of heat map

marker_layer = gmaps.marker_layer(locations,info_box_content = hotel_info)
fig.add_layer(marker_layer)


# Display figure
fig

Figure(layout=FigureLayout(height='420px'))